<a href="https://colab.research.google.com/github/Hambeurger/Deep-Learning-Project-2/blob/main/Deep_Learning_Project_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all the libraries.

In [1]:
import urllib.request
import zipfile
import pickle as pk
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
import numpy as np

1. Data Preparation: Download and Load the Data.

The data is a 2D numpy array, where each vector is of size 1000.

In [3]:
zip_url = "https://github.com/Hambeurger/Deep-Learning-Project-2/raw/main/input_data.zip"
zip_path, _ = urllib.request.urlretrieve(zip_url)
target_folder = "temp"

# Unzipping the downloaded file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(target_folder)

# Loading the data
data_path = f"{target_folder}/input_data.pkl"
with open(data_path, 'rb') as f:
    dd = pk.load(f)
data = dd['data']  # Data has a shape of (150, 1000)

2. Dataset Preparation

Generate corrupted versions of the input data to train the denoising autoencoder.

In [4]:
class CustomDataset(Dataset):
    def __init__(self, data):
        self.data = data

    # Randomly corrupt the input by flipping or altering some values
    def corrupt_vector(self, vector):

        corruption_type = np.random.choice(['flip', 'alter'], p=[0.5, 0.5])
        corrupted_vector = np.copy(vector)

        if corruption_type == 'flip':
            flip_indices = np.random.choice(len(vector), size=int(0.1 * len(vector)), replace=False)
            corrupted_vector[flip_indices] *= -1
        elif corruption_type == 'alter':
            alter_indices = np.random.choice(len(vector), size=int(0.1 * len(vector)), replace=False)
            corrupted_vector[alter_indices] = np.random.uniform(-1, 1, size=alter_indices.shape)

        return corrupted_vector

    def __len__(self):
        return len(self.data)

    # Returns both the corrupted vector (input to the model) and the original vector (target output)
    def __getitem__(self, idx):
        original_vector = self.data[idx]
        corrupted_vector = self.corrupt_vector(original_vector)
        return torch.tensor(corrupted_vector, dtype=torch.float32), torch.tensor(original_vector, dtype=torch.float32)

# Create dataset and DataLoader
dataset = CustomDataset(data)
dataloader = DataLoader(dataset, batch_size=16, shuffle=True)

3. Model Architecture - Denoising Autoencoder

A denoising autoencoder is built using fully connected layers. The encoder compresses the input into a lower-dimensional representation (128 neurons in the bottleneck), and the decoder reconstructs it back to the original dimension (1000 neurons).

In [5]:
class DenoisingAutoencoder(nn.Module):
    def __init__(self, input_size=1000, hidden_size=512):
        super(DenoisingAutoencoder, self).__init__()

        # Encoder
        self.encoder = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, 128),
            nn.ReLU()
        )

        # Decoder
        self.decoder = nn.Sequential(
            nn.Linear(128, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, input_size)
        )

    def forward(self, x):
        # Encode and decode process
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)

        # Apply sign function to ensure output in {-1, 1}
        return torch.sign(decoded)

# Instantiate the model
model = DenoisingAutoencoder()

4. Training process

The model is trained using Mean Squared Error (MSE) loss, which measures the reconstruction error between the original and reconstructed vectors. The Adam optimizer is used to update the weights. The network is trained for 50 epochs, printing the average loss at the end of each epoch.

In [6]:
# Training setup
criterion = nn.MSELoss()  # Mean Squared Error loss to minimize reconstruction error
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 50

for epoch in range(num_epochs):
    total_loss = 0.0
    for corrupted_vectors, original_vectors in dataloader:
        # Zero the gradient buffers
        optimizer.zero_grad()

        # Forward pass: Get model outputs
        outputs = model(corrupted_vectors)

        # Calculate the loss
        loss = criterion(outputs, original_vectors)

        # Backpropagate the loss
        loss.backward()

        # Update the weights
        optimizer.step()

        total_loss += loss.item()

    avg_loss = total_loss / len(dataloader)
    print(f'Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.4f}')

Epoch [1/50], Loss: 1.9975
Epoch [2/50], Loss: 2.0015
Epoch [3/50], Loss: 2.0020
Epoch [4/50], Loss: 2.0027
Epoch [5/50], Loss: 2.0019
Epoch [6/50], Loss: 2.0046
Epoch [7/50], Loss: 1.9984
Epoch [8/50], Loss: 1.9985
Epoch [9/50], Loss: 2.0010
Epoch [10/50], Loss: 2.0021
Epoch [11/50], Loss: 2.0088
Epoch [12/50], Loss: 2.0028
Epoch [13/50], Loss: 2.0079
Epoch [14/50], Loss: 2.0025
Epoch [15/50], Loss: 1.9988
Epoch [16/50], Loss: 2.0000
Epoch [17/50], Loss: 2.0007
Epoch [18/50], Loss: 2.0015
Epoch [19/50], Loss: 2.0055
Epoch [20/50], Loss: 2.0054
Epoch [21/50], Loss: 2.0056
Epoch [22/50], Loss: 2.0031
Epoch [23/50], Loss: 2.0020
Epoch [24/50], Loss: 2.0018
Epoch [25/50], Loss: 2.0036
Epoch [26/50], Loss: 1.9998
Epoch [27/50], Loss: 2.0074
Epoch [28/50], Loss: 2.0022
Epoch [29/50], Loss: 2.0023
Epoch [30/50], Loss: 1.9978
Epoch [31/50], Loss: 1.9969
Epoch [32/50], Loss: 2.0067
Epoch [33/50], Loss: 2.0033
Epoch [34/50], Loss: 2.0035
Epoch [35/50], Loss: 2.0067
Epoch [36/50], Loss: 2.0049
E

5. Evaluation of the model

After training, the model is evaluated using the same MSE loss function, gradient updates are disabled (with torch.no_grad()) to speed up evaluation and save memory. Measure accuracy to ensure exact values in {-1,1}.

In [7]:
def evaluate_model(model, dataset, criterion):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0.0
    total_accuracy = 0.0
    total_samples = 0
    with torch.no_grad():  # Disable gradient computation for evaluation
        for corrupted_vectors, original_vectors in dataset:
            outputs = model(corrupted_vectors)
            loss = criterion(outputs, original_vectors)
            total_loss += loss.item()

            # Accuracy: Compare the output with original
            predicted = torch.sign(outputs)
            correct_matches = (predicted == original_vectors).sum().item()
            total_accuracy += correct_matches
            total_samples += original_vectors.numel()  # Total number of elements

    avg_loss = total_loss / len(dataset)
    accuracy = total_accuracy / total_samples
    return avg_loss, accuracy

# Model evaluation on the dataset
test_loss, accuracy = evaluate_model(model, dataset, criterion)
print(f'Test Loss (Reconstruction Error): {test_loss:.4f}')
print(f'Accuracy (Exact Recovery): {accuracy:.4%}')

Test Loss (Reconstruction Error): 1.9998
Accuracy (Exact Recovery): 50.0060%


6. Model saving

The trained model's parameters are saved in a file denoising_autoencoder_model.pth for future use.

In [8]:
torch.save(model.state_dict(), 'denoising_autoencoder_model.pth')